In [2]:
from skimage import filters
from skimage import feature
from skimage.filters import prewitt_h,prewitt_v
from skimage.io import imread, imshow
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
%pip install -U git+https://github.com/szagoruyko/pytorchviz.git@master
from torchviz import make_dot, make_dot_from_trace
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import time
import torch
import imghdr
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from itertools import chain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/szagoruyko/pytorchviz.git (to revision master) to /tmp/pip-req-build-h1bnq39n
  Running command git clone -q https://github.com/szagoruyko/pytorchviz.git /tmp/pip-req-build-h1bnq39n
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
parent=r'C:\Users\Abdul Wahid Awan\Downloads'

In [8]:
featured_path=parent+'\Copy\FeaturedImages'
# featured_path ='/content/drive/MyDrive/FeaturedImages'

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


# Glove 6B

In [4]:
glove_path="/content/drive/MyDrive/Glove_6B"
# glove_path=parent+"\Glove6B"
os.chdir(glove_path)

In [5]:
def glove_model(glove_path):
    glove_dict = {}
    with open(glove_path+'/'+'glove.6B.300d.txt','r',encoding="utf8") as f:
        for line in f:
            string = line.split()
            word = string[0]
            feature = np.array(string[1:], dtype=np.float64)
            glove_dict[word] = feature
    return glove_dict

In [6]:
glove=glove_model(glove_path)

# All Images DataFrame

In [ ]:
os.chdir(r"C:\Users\Abdul Wahid Awan\Downloads\Copy")
df=pd.read_csv("ALL.csv")
df.loc[df['overall_sentiment'] =='0' , 'overall_sentiment'] = 1
df.loc[df['overall_sentiment'] =='1' , 'overall_sentiment'] = 2
df.loc[df['overall_sentiment'] =='2' , 'overall_sentiment'] = 0

df.loc[df['humour'] =='funny' , 'humour'] = 1
df.loc[df['humour'] =='very_funny' , 'humour'] = 1
df.loc[df['humour'] =='hilarious' , 'humour'] = 1
df.loc[df['humour'] =='not_funny' , 'humour'] = 0


df.loc[df['sarcasm'] =='general' , 'sarcasm'] = 1
df.loc[df['sarcasm'] =='twisted_meaning' , 'sarcasm'] = 1
df.loc[df['sarcasm'] =='very_twisted' , 'sarcasm'] = 1
df.loc[df['sarcasm'] =='not_sarcastic' , 'sarcasm'] = 0


df.loc[df['offensive'] =='offensive' , 'offensive'] = 1
df.loc[df['offensive'] =='slight' , 'offensive'] = 1
df.loc[df['offensive'] =='hateful_offensive' , 'offensive'] = 1
df.loc[df['offensive'] =='very_offensive' , 'offensive'] = 1
df.loc[df['offensive'] =='not_offensive' , 'offensive'] = 0

df.loc[df['motivational'] =='not_motivational' , 'motivational'] = 0
df.loc[df['motivational'] =='motivational' , 'motivational'] = 1

In [ ]:
df.to_csv('ALLnew.csv')

In [7]:
os.chdir('/content/drive/MyDrive/Project_AI')
df=pd.read_csv("ALLnew.csv")
df=df.drop(labels="Unnamed: 0.1",axis=1)
df=df.drop(labels="Unnamed: 0",axis=1)

In [33]:
df

,image_name,text_ocr,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,1,1,0,0,0
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,0,1,0,1,0
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,1,0,0,0,0
3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,1,1,1,1,0
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,1,1,1,0,2
...,...,...,...,...,...,...,...,...
12948,image_6961_rotated180FLR.jpeg,THAT FACE YOUR FRIEND GIVES YOU WHEN YOUR CRUS...,THAT FACE YOUR FRIEND GIVES YOU WHEN YOUR CRUS...,1,1,1,0,1
12949,image_6962_rotated180FLR.jpeg,NIGHTMARES Are made of this.,NIGHTMARES Are made of this.,0,0,0,1,1
12950,image_6966_rotated180FLR.png,Baby Mr. Bean Funny Picture,Baby Mr. Bean Funny Picture,1,1,1,0,1
12951,image_6972_rotated180FLR.jpeg,YOUR FACE MemeCenter.com WHEN THERES A SUBSTIT...,YOUR FACE MemeCenter.com WHEN THERES A SUBSTIT...,1,1,1,0,1


# Features 

In [9]:
def image_features(img):
    os.chdir(featured_path)
    image1 =imread(img,as_gray=True)
    feature =np.array(image1,dtype='float32')
    return feature


In [10]:
def text_features(img):
    os.chdir(featured_path)
    feature=[]
    line=df[df['image_name'] ==img]['text_corrected'].iloc[0]
    text=line
    text=text.lower()
    words=word_tokenize(text)
    for j in words:
        try:
            feature.append(glove[j])
        except:
            pass
    feature=np.array(feature).flatten()
    maxi=62100
    feature=np.pad(feature,(0,maxi-len(feature)))
    return feature

# Data Loader

In [11]:
class Data(Dataset):
    def __init__(self,df,train,test):
        self.all_images = (np.array(df['image_name'])).tolist()
        self.labels = (np.array(df[['humour','sarcasm','offensive','motivational']])).tolist()
        self.train_images = int((len(df)*0.8))
        self.valid_ratio = len(df) - self.train_images
       
        
        if train == True:
            self.images = self.all_images[:self.train_images]
            self.label = self.labels[:self.train_images]

            self.transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((270, 275)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomRotation(degrees=90),
                transforms.ToTensor(),
            ])
            
        elif test==True:
            self.images = list(self.all_images[-self.valid_ratio:])
            self.label = list(self.labels[-self.valid_ratio:])
            
            self.transform = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((270, 275)),
                transforms.ToTensor(),
            ])
           
            
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, position):
        tf= text_features(self.images[position])
        img = image_features(self.images[position])
        img = self.transform(img)
        img = img.flatten()
        outputlabel = self.label[position]
       
        outputlabel= torch.tensor(outputlabel, dtype=torch.float32)
        img= torch.tensor(img, dtype=torch.float32)
        tf=  torch.tensor(tf, dtype=torch.float32)
        dict1= {"Image":img,"Text":tf,"OutputLabel":outputlabel}

        return dict1

# Neural Network

In [12]:
class Multi_Model_NN(nn.Module):
    def __init__(self,textinput,inputneurons,textH1,textH2,comH1,ImageH1,ImageH2,all4,ImageH3,textH3,ImageH4,comH2,comH3,sh1,sh2,Hh1,Hh2,mh1,mh2,oh1,oh2,outall):
        super(Multi_Model_NN, self).__init__()
        
        self.text = nn.Sequential(
            nn.Linear(textinput, textH1),
            nn.ReLU(),
            nn.Linear(textH1, textH2),
            nn.ReLU(),
            nn.Linear(textH2, textH3)
       )
        self.image = nn.Sequential(
            nn.Linear(inputneurons,ImageH1),
            nn.Sigmoid(),
            nn.Linear(ImageH1, ImageH2),
            nn.Sigmoid(),
            nn.Linear(ImageH2, ImageH3),
            nn.ReLU(),
            nn.Linear(ImageH3, ImageH4)
        )
        
        self.combination_of_models = nn.Sequential(
            nn.Linear(textH3+ImageH4,comH1),
            nn.ReLU(),
            nn.Linear(comH1,comH2),
            nn.Sigmoid(),
            nn.Linear(comH2,comH3),
            nn.ReLU(),
            nn.Linear(comH3,all4)
        )
        
        self.sarcasm = nn.Sequential(
            nn.Linear(all4, sh1),
            nn.ReLU(),
            nn.Linear(sh1, sh2),
            nn.ReLU(),
            nn.Linear(sh2, outall),
        )
        
        self.humour = nn.Sequential(
            nn.Linear(all4, Hh1),
            nn.ReLU(),
            nn.Linear(Hh1, Hh2),
            nn.ReLU(),
            nn.Linear(Hh2, outall),
        )

        self.motivational = nn.Sequential(
            nn.Linear(all4, mh1),
            nn.ReLU(),
            nn.Linear(mh1, mh2),
            nn.ReLU(),
            nn.Linear(mh2, outall),
        )

        self.offensive = nn.Sequential(
            nn.Linear(all4, oh1),
            nn.ReLU(),
            nn.Linear(oh1, oh2),
            nn.ReLU(),
            nn.Linear(oh2, outall),
        )

    def forward(self, img, text):
        img_output = self.image(img)
        t_output = self.text(text)

        dual = torch.cat([t_output,img_output],1)
        x = self.combination_of_models(dual)
        
        o_out = self.offensive(x)
        m_out = self.motivational(x)
        h_out = self.humour(x)
        s_out = self.sarcasm(x)
        
        return o_out,m_out,h_out,s_out

In [13]:
imginputneurons=74250
imghidden1=1000
imghidden2=800
imghidden3=600
imghidden4=400


textinput=62100
texthidden1=1000
texthidden2=700
texthidden3=400


comh1=700
comh2=650
comh3=600

all4 =500
sh1 = 300
sh2 = 200

Hh1= 400
Hh2= 200

mh1= 400
mh2= 200

oh1= 400 
oh2= 200
outall=2

neural_network=Multi_Model_NN(textinput,imginputneurons,texthidden1,texthidden2,comh1,imghidden1,imghidden2,all4,imghidden3,texthidden3,imghidden4,comh2,comh3,sh1,sh2,Hh1,Hh2,mh1,mh2,oh1,oh2,outall)   
print(neural_network.parameters)    

<bound method Module.parameters of Multi_Model_NN(
  (text): Sequential(
    (0): Linear(in_features=62100, out_features=1000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1000, out_features=700, bias=True)
    (3): ReLU()
    (4): Linear(in_features=700, out_features=400, bias=True)
  )
  (image): Sequential(
    (0): Linear(in_features=74250, out_features=1000, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=1000, out_features=800, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=800, out_features=600, bias=True)
    (5): ReLU()
    (6): Linear(in_features=600, out_features=400, bias=True)
  )
  (combination_of_models): Sequential(
    (0): Linear(in_features=800, out_features=700, bias=True)
    (1): ReLU()
    (2): Linear(in_features=700, out_features=650, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=650, out_features=600, bias=True)
    (5): ReLU()
    (6): Linear(in_features=600, out_features=500, bias=True)
  )
  (sarcasm): Sequential(
 

In [ ]:
%%time
training_data = Data(df, train=True, test=False)
loader = DataLoader(training_data, batch_size=60,shuffle=True)

criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(neural_network.parameters(), lr=0.001)
llist = []
epoches=8

for x in range(epoches):
    print("Epoches: ",x+1)
    for key,value in enumerate(loader):
        y_pred = neural_network(value['Image'],value["Text"])
        loss = 0
        
        for i in range(0,len(y_pred)):
            loss += criterion(y_pred[i],torch.transpose(value['OutputLabel'],1,0)[i].long()) 
        
        llist.append(loss.item())
        optimizer.zero_grad()
        loss.backward()  
        optimizer.step() 

Epoches:  1
Epoches:  2
Epoches:  3
Epoches:  4
Epoches:  5
Epoches:  6
Epoches:  7
Epoches:  8


# Saving the Model

In [ ]:
model_path=parent+"\model_4labels.pth"

In [ ]:
torch.save(neural_network.state_dict(), model_path)

# Loading Model

In [ ]:
multimodel =Multi_Model_NN(textinput,imginputneurons,texthidden1,texthidden2,comh1,imghidden1,imghidden2,all4,imghidden3,texthidden3,imghidden4,comh2,comh3,sh1,sh2,Hh1,Hh2,mh1,mh2,oh1,oh2,outall)     
multimodel.load_state_dict(torch.load(model_path, map_location='cpu'))
multimodel.eval()

Multi_Model_NN(
  (text): Sequential(
    (0): Linear(in_features=62100, out_features=1000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1000, out_features=700, bias=True)
    (3): ReLU()
    (4): Linear(in_features=700, out_features=400, bias=True)
  )
  (image): Sequential(
    (0): Linear(in_features=74250, out_features=1000, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=1000, out_features=800, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=800, out_features=600, bias=True)
    (5): ReLU()
    (6): Linear(in_features=600, out_features=400, bias=True)
  )
  (combination_of_models): Sequential(
    (0): Linear(in_features=800, out_features=700, bias=True)
    (1): ReLU()
    (2): Linear(in_features=700, out_features=650, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=650, out_features=600, bias=True)
    (5): ReLU()
    (6): Linear(in_features=600, out_features=500, bias=True)
  )
  (sarcasm): Sequential(
    (0): Linear(in_features=500, out

# Architecture

In [50]:
images=np.array(df["image_name"])
os.chdir(featured_path)

temp = Image.open(images[0])
temp= temp.convert('L')
temp = temp.resize((270,275))

feature =np.array(temp,dtype='float32' )
feature=feature.flatten()
feature=torch.tensor(feature,dtype=torch.float32)

t_feature=text_features(images[0])
t_feature= torch.tensor(t_feature,dtype=torch.float32)

make_dot(neural_network(feature.unsqueeze(0),t_feature.unsqueeze(0)), params=dict(neural_network.named_parameters())).render("i20_2367_Model_Part2","png")

'png/i20_2367_Model_Part2.pdf'

# Testing data

In [31]:
testing_data = Data(df, train=False, test=True)
testloader = DataLoader(testing_data, batch_size=60,shuffle=True)

llist = []
predict=[]
actual=[]

for key,value in enumerate(testloader):
    y_pred = multimodel(value['Image'],value["Text"])
    loss = 0
    for i in range(0,len(y_pred)):
        loss += criterion(y_pred[i],torch.transpose(value['OutputLabel'],1,0)[i].long()) 
        actual.append(torch.transpose(value['OutputLabel'],1,0)[i].tolist())
    
    llist.append(loss.item())
    for i in range(0,len(y_pred)):
        predict.append(torch.argmax(y_pred[i], dim=1).tolist())
    
   


In [ ]:
actual = list(chain.from_iterable(actual))
predict = list(chain.from_iterable(predict))

# Accuracy and F1Score

In [ ]:
from sklearn.metrics import f1_score,accuracy_score
print("F1Score: ",f1_score(actual,predict,average='macro')*100)
print("Accuracy: ",accuracy_score(actual,predict)*100)


F1Score:  66.2664101154715
Accuracy:  70.55191045928213
